In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/25 22:47:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Question 1
spark.version

'3.0.3'

In [5]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-22 17:19:48--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.172.25
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.172.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  34.7MB/s    in 23s     

2022-02-22 17:20:11 (30.7 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [22]:
df = spark.read \
     .option("header", "true") \
     .csv('fhvhv_tripdata_2021-02.csv')

In [23]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [62]:
from pyspark.sql import types

In [63]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [26]:
df = spark.read \
     .option("header", "true") \
     .schema(schema) \
     .csv('fhvhv_tripdata_2021-02.csv')

In [27]:
df = df.repartition(24)

In [28]:
df.write.parquet("code/fhvhv/2021/02", mode="overwrite")

In [3]:
df = spark.read.parquet("code/fhvhv/2021/02")

In [4]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [5]:
from pyspark.sql import functions as F

In [67]:
# QUESTION 3
df \
    .select('hvfhs_license_num', 'pickup_datetime') \
    .filter(F.to_date(df.pickup_datetime) == '2021-02-15') \
    .count()

367170

In [43]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [53]:
# QUESTION 4
df \
    .withColumn('duration_minutes', ((F.unix_timestamp(df.dropoff_datetime) - F.unix_timestamp(df.pickup_datetime)) / 60)) \
    .select('hvfhs_license_num','duration_minutes','pickup_datetime', 'dropoff_datetime') \
    .sort(F.desc('duration_minutes')) \
    .show()

+-----------------+------------------+-------------------+-------------------+
|hvfhs_license_num|  duration_minutes|    pickup_datetime|   dropoff_datetime|
+-----------------+------------------+-------------------+-------------------+
|           HV0005|            1259.0|2021-02-11 13:40:44|2021-02-12 10:39:44|
|           HV0004| 953.6833333333333|2021-02-17 15:54:53|2021-02-18 07:48:34|
|           HV0004| 733.9833333333333|2021-02-20 12:08:15|2021-02-21 00:22:14|
|           HV0003|            677.55|2021-02-03 20:24:25|2021-02-04 07:41:58|
|           HV0003| 626.2833333333333|2021-02-19 23:17:44|2021-02-20 09:44:01|
|           HV0003|             583.5|2021-02-25 17:13:35|2021-02-26 02:57:05|
|           HV0003|             580.1|2021-02-20 01:36:13|2021-02-20 11:16:19|
|           HV0005| 576.8666666666667|2021-02-18 15:24:19|2021-02-19 01:01:11|
|           HV0003| 575.9166666666666|2021-02-18 01:31:20|2021-02-18 11:07:15|
|           HV0005| 569.4833333333333|2021-02-10 20:

In [6]:
# QUESTION 5
df.registerTempTable('fhvhv_data')

In [7]:
# QUESTION 5
spark.sql("""
SELECT dispatching_base_num,
       count(1) as count
FROM
    fhvhv_data
GROUP BY
    dispatching_base_num
ORDER BY count desc
""").show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [73]:
# QUESTION 6
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-02-22 18:36:02--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.80.94
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.80.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-02-22 18:36:02 (120 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [75]:
df_zones = spark.read \
     .option("header", "true") \
     .csv('taxi+_zone_lookup.csv')

In [76]:
df_zones.schema

StructType(List(StructField(LocationID,StringType,true),StructField(Borough,StringType,true),StructField(Zone,StringType,true),StructField(service_zone,StringType,true)))

In [77]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [78]:
schema_zones = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [79]:
df_zones = spark.read \
     .option("header", "true") \
     .schema(schema_zones) \
     .csv('taxi+_zone_lookup.csv')

In [80]:
df_zones.schema

StructType(List(StructField(LocationID,IntegerType,true),StructField(Borough,StringType,true),StructField(Zone,StringType,true),StructField(service_zone,StringType,true)))

In [81]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [83]:
df_zones.write.parquet("code/zones/", mode="overwrite")

In [8]:
df_zones = spark.read.parquet("code/zones")

In [9]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [10]:
df_zones.registerTempTable('zones')

In [24]:
spark.sql("""
SELECT
       zpu.Zone as puZone,
       zdo.Zone as doZone,
       CONCAT(zpu.Zone, ' / ', zdo.Zone) as pair
FROM fhvhv_data
FULL OUTER JOIN zones as zpu ON fhvhv_data.PULocationID=zpu.LocationID
FULL OUTER JOIN zones as zdo ON fhvhv_data.DOLocationID=zdo.LocationID
ORDER BY fhvhv_data.dispatching_base_num
LIMIT 100
""").show(100,truncate=False)

+---------------------------------------------+---------------------------------------------+------------------------------------------------------------+
|puZone                                       |doZone                                       |pair                                                        |
+---------------------------------------------+---------------------------------------------+------------------------------------------------------------+
|null                                         |Rikers Island                                |null                                                        |
|Governor's Island/Ellis Island/Liberty Island|null                                         |null                                                        |
|null                                         |NV                                           |null                                                        |
|null                                         |Governor's Island/Ellis

In [15]:
spark.sql("""
SELECT CONCAT(zpu.Zone, ' / ', zdo.Zone) as pair, count(1) as count
FROM fhvhv_data
FULL OUTER JOIN zones as zpu ON fhvhv_data.PULocationID=zpu.LocationID
FULL OUTER JOIN zones as zdo ON fhvhv_data.DOLocationID=zdo.LocationID
GROUP BY pair
ORDER BY count desc
""").show(200,truncate=False)

+---------------------------------------------------------------------+-----+
|pair                                                                 |count|
+---------------------------------------------------------------------+-----+
|East New York / East New York                                        |45041|
|Borough Park / Borough Park                                          |37329|
|Canarsie / Canarsie                                                  |28026|
|Crown Heights North / Crown Heights North                            |25976|
|Bay Ridge / Bay Ridge                                                |17934|
|Astoria / Astoria                                                    |14688|
|Jackson Heights / Jackson Heights                                    |14688|
|Central Harlem North / Central Harlem North                          |14481|
|Bushwick South / Bushwick South                                      |14424|
|Flatbush/Ditmas Park / Flatbush/Ditmas Park                    